### TKOH Sample Code

In [1]:
# dependencies
import pandas as pd
import json
import math

In [2]:
# load beacon data
raw_data = pd.read_csv('survey/0621/beacon.csv')
raw_data.columns = ['gateway', 'rssi', 'ts', 'raw_ble']
raw_data

,gateway,rssi,ts,raw_ble
0,BL-43,-86,1.655801e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
1,BL-42,-82,1.655801e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
2,BL-43,-88,1.655801e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
3,BL-42,-80,1.655801e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
4,BL-43,-85,1.655801e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
...,...,...,...,...
38519,BL-42,-78,1.655803e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
38520,BL-43,-83,1.655803e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
38521,BL-43,-83,1.655803e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...
38522,BL-42,-85,1.655803e+09,043E2A02010001C0BBC40423ED1E0201061AFF4C000215...


In [3]:
# load ground truth records
gt =  open('survey/0621/gt-8F.txt', 'r')
# gt =  open('survey/0621/gt-G.txt', 'r')
# gt =  open('survey/0621/gt-LG.txt', 'r')
gt = json.loads(gt.read())

print(f"Read Ground-Truth --- Event:{gt['event'][0]['event']}  Date:{gt['event'][0]['date']}  Remark{gt['event'][0]['remark']}")

check_point = json.loads(gt['event_plan'][0]['geojson'])
check_time = json.loads(gt['event_plan'][0]['ground_truth'])

def asRadians(degrees):
    return degrees * math.pi / 180

def getXYpos(rlat, rlog, plat, plong):
    """ Calculates X and Y distances in meters.
    """
    deltaLatitude = plat - rlat
    deltaLongitude = plong - rlog
    latitudeCircumference = 40075160 * math.cos(asRadians(rlat))
    resultX = deltaLongitude * latitudeCircumference / 360
    resultY = deltaLatitude * 40008000 / 360
    return resultX, resultY


idx = 0
g2m = 111194.926644

while idx < len(check_time):
    point_data = raw_data[(raw_data.ts > check_time[idx][0]) & (raw_data.ts < check_time[idx][1])]
    print(f'Coordinate:{check_point[idx]} Time-Range{check_time[idx]} Data length:{point_data.shape}')
    idx += 1

Read Ground-Truth --- Event:1655784581  Date:2022-06-21  Remark8F
Coordinate:[114.27012436079315, 22.317768396149717] Time-Range[1655801554.664, 1655801809.025] Data length:(2590, 4)
Coordinate:[114.27019610382814, 22.317756105111542] Time-Range[1655801827.414, 1655802094.486] Data length:(5328, 4)
Coordinate:[114.2702244779922, 22.317800221130014] Time-Range[1655802107.222, 1655802319.871] Data length:(4827, 4)
Coordinate:[114.27026706610701, 22.31783471453292] Time-Range[1655802359.933, 1655802378.437] Data length:(432, 4)
Coordinate:[114.27031461618446, 22.317792692764712] Time-Range[1655802387.714, 1655802617.142] Data length:(2775, 4)
